In [ ]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm,trange
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import pickle
from utils import *

plt.rcParams['font.sans-serif'] = ['KaiTi'] # 指定默认字体
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
%config InlineBackend.figure_format = 'retina'

## 获取所有交易日、价量数据和股票基本信息数据

In [ ]:
tradedays_list = list(np.sort(pd.read_excel('data\\stock_index\\shse.xlsx').iloc[:,0].values))

In [ ]:
## 注：价量数据里只有一支北交所股票，没有新三板股票，并且价量数据已经经过预处理，日期是连续的
data = pd.read_pickle('data\\AShareEODPrices.pkl')
data = data.sort_values(['code','date']).reset_index(drop=True)
data.index = pd.MultiIndex.from_arrays([data.code.values, data.date.values], names=['code','date'])
data = data.iloc[:,2:]
data['returns'] = data['adjclose']/data['adjpreclose']-1+data['volume']-data['volume']

In [ ]:
## 获取Asharedescription
## 注：以A开头的股票代码是上市失败的股票；Asharedescription不含新三板股票
Asharedescription = pd.read_csv('data\\AShareDescription.csv',encoding='gbk')
Asharedescription['S_INFO_LISTDATE'] = pd.to_datetime(Asharedescription['S_INFO_LISTDATE']).apply(lambda x: x.strftime('%Y-%m-%d'))
Asharedescription.index = Asharedescription['S_INFO_WINDCODE'].values

## 把Asharedescription的ONOFFHATDATE列变成ST的时间区间，格式形如[(start1,end1),(start2,end2),...]
def f(temp, latest_day=tradedays_list[-1]):
    if pd.isna(temp):
        return []
    temp = np.array(temp.split(',')[::-1])
    ST_end_index = np.array([], dtype=np.int64)
    for i in range(len(temp)):
        if '去ST' in temp[i]:
            ST_end_index = np.append(ST_end_index,[i])
    ST_start_index = np.append([0],ST_end_index+1)
    if ST_start_index[-1]==len(temp):
        ST_start_index = ST_start_index[:-1]
    for i in range(len(temp)):
        temp[i] = pd.to_datetime(re.search('\d+',temp[i]).group()).strftime('%Y-%m-%d')
    ST_range = []
    for i in range(len(ST_end_index)):
        ST_range.append((temp[ST_start_index[i]],temp[ST_end_index[i]]))
    if len(ST_start_index)>len(ST_end_index):
        ST_range.append((temp[ST_start_index[-1]],latest_day))
    return ST_range
Asharedescription['ONOFFHATDATE'] = Asharedescription['ONOFFHATDATE'].apply(lambda x: f(x))

Asharedescription.to_pickle('data preprocess\\Asharedescription.pkl')

## 提取股权激励基本信息数据

### 从AShareIncDescription提取数据

In [ ]:
## 获取基本的股权激励数据Inc_data
data_temp = pd.read_pickle('data\\AShareIncDescription.pkl')

## 注：根据(S_INFO_WINDCODE、S_INC_SEQUENCE)判断没有重复行,即完全可以根据S_INFO_WINDCODE、S_INC_SEQUENCE作为Inc_data的索引;但是无法根据（S_INFO_WINDCODE、PREPLAN_ANN_DATE）唯一确定AShareIncDescription的某行，这说明有的公司会在同一天发两个股权激励预案，这在后面设计策略时要注意
Inc_data = data_temp[['S_INFO_WINDCODE', 'S_INC_SEQUENCE', 'PREPLAN_ANN_DATE', 'GM_DATE', 'IMPLEMENT_DATE', 'INTERVAL_MONTHS', 
                    'S_INC_SUBJECT', 'S_INC_TYPE', 'INC_NUMBERS_RATE', 'S_INC_INITEXECPRI',
                    'PROGRESS', 'S_INC_EXPIRYDATE']].sort_values('PREPLAN_ANN_DATE').reset_index(drop=True)

## 得到PREPLAN_ANN_DATE_to_trade和PREPLAN_ANN_DATE_to_trade_next
Inc_data['PREPLAN_ANN_DATE'] = pd.to_datetime(Inc_data['PREPLAN_ANN_DATE']).apply(lambda x:str(x)[:10])
Inc_data['GM_DATE'] = pd.to_datetime(Inc_data['GM_DATE']).apply(lambda x:str(x)[:10])
Inc_data['IMPLEMENT_DATE'] = pd.to_datetime(Inc_data['IMPLEMENT_DATE']).apply(lambda x:str(x)[:10])

Inc_data['PREPLAN_ANN_DATE_to_trade_next'] = [to_tradeday(i, tradedays_list) for i in Inc_data['PREPLAN_ANN_DATE']]
Inc_data['PREPLAN_ANN_DATE_to_trade'] = [to_tradeday(i, tradedays_list, n=0) for i in Inc_data['PREPLAN_ANN_DATE']]
Inc_data = Inc_data.sort_values('PREPLAN_ANN_DATE').reset_index(drop=True)
Inc_data.index = pd.MultiIndex.from_arrays(Inc_data[['S_INFO_WINDCODE','S_INC_SEQUENCE']].values.T)

#### 插入论文里要用到的图片和数据

In [ ]:
## A股市场各年股权激励的预案公告数目
temp = Inc_data['PREPLAN_ANN_DATE'].apply(lambda x: x[:4])
temp = temp.groupby(temp).count()
temp.index = list(temp.index[:-1])+[Inc_data['PREPLAN_ANN_DATE'].iloc[-1]]
temp.plot(kind='bar', title='A股市场各年股权激励的预案公告数目')
plt.savefig('show_data\\引言\\A股市场各年股权激励的预案公告数目.png', dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
## 成功实施的比例
print((Inc_data.PROGRESS=='3').sum())
print((Inc_data.PROGRESS=='3').sum()/Inc_data.shape[0])

### 剔除Inc_data上市不满一年的行、剔除Inc_data里预案公告日比Asharedescription里上市日还靠前的股票（有些是之前新三板里的，有些是根本没上市）

In [ ]:
## 剔除Inc_data里预案公告日比Asharedescription里上市日还靠前的股票
Inc_data = Inc_data[Asharedescription.loc[list(Inc_data['S_INFO_WINDCODE'].values)]['S_INFO_LISTDATE'].values<Inc_data['PREPLAN_ANN_DATE'].values].reset_index(drop=True)
## 剔除股权激励数据包含但价量数据不包含的公司对应的行
temp = list(set(Inc_data['S_INFO_WINDCODE'])-set(data.index.get_level_values(0)))
Inc_data = see_row(Inc_data, 'S_INFO_WINDCODE', list(temp), reverse=True).reset_index(drop=True)
## 剔除Inc_data上市不满一年的行和退市后的行
def f(x, Asharedescription_temp):
    PREPLAN_ANN_DATE = x.loc['PREPLAN_ANN_DATE']
    PREPLAN_ANN_DATE_one_year_before = str(int(PREPLAN_ANN_DATE[:4])-1)+PREPLAN_ANN_DATE[4:]
    LISTDATE = pd.to_datetime(str(Asharedescription_temp['S_INFO_LISTDATE'])).strftime('%Y-%m-%d')
    DELISTDATE = Asharedescription_temp['S_INFO_DELISTDATE']
    if not pd.isna(DELISTDATE):
        DELISTDATE = pd.to_datetime(str(DELISTDATE)).strftime('%Y-%m-%d')
    if LISTDATE<=PREPLAN_ANN_DATE_one_year_before:
        if pd.isna(DELISTDATE) or DELISTDATE>PREPLAN_ANN_DATE:
            return True
    return False
bools = Inc_data.apply(lambda x: f(x,Asharedescription.loc[x['S_INFO_WINDCODE']]),axis=1)
Inc_data = Inc_data[bools].reset_index(drop=True)
Inc_data.index = pd.MultiIndex.from_arrays(Inc_data[['S_INFO_WINDCODE','S_INC_SEQUENCE']].values.T)

In [ ]:
Inc_data.to_pickle('data preprocess\\Inc_data.pkl')

## 提取文本信息

In [ ]:
Inc_text_data = data_temp[['S_INFO_WINDCODE', 'S_INC_SEQUENCE', 'PREPLAN_ANN_DATE', 
                            'S_INC_INCENTCONDITION', 'S_INC_INCENTSHARESALEDESCRIPT', 'PRICE_DESCRIPTION', 
                            'S_INC_PROGRAMDESCRIPT', 'S_INC_OPTEXESPECIALCONDITION']].copy()
Inc_text_data['PREPLAN_ANN_DATE'] = pd.to_datetime(Inc_text_data['PREPLAN_ANN_DATE']).apply(lambda x:str(x)[:10])
Inc_text_data['PREPLAN_ANN_DATE_to_trade'] = [to_tradeday(i, tradedays_list, n=0) for i in Inc_text_data['PREPLAN_ANN_DATE']]
Inc_text_data = Inc_text_data.sort_values('PREPLAN_ANN_DATE').reset_index(drop=True)
Inc_text_data.index = pd.MultiIndex.from_frame(Inc_text_data[['S_INFO_WINDCODE', 'S_INC_SEQUENCE']])
Inc_text_data.index.names = [None, None]
Inc_text_data = Inc_text_data.loc[Inc_data.index]

#### 提取行权/授予价格信息

In [ ]:
## 注：Inc_data里的行权价格是最新调整后的数据；Inc_text_data里的PRICE_DESCRIPTION大部分包含历史调整的情况，但部分包含的不全，无法找到预案时的的行权价格
## 不过缺失的不多，46个
## 注：有的股权激励有两个激励价格，这种情况只能记录第一个，因为无法直接得到两种价格的股票比例
def f(x):
    if pd.isna(x):
        return np.nan
    x = re.sub('[【】 ]','',x)
    ## 如果以数字开始，且以数字结尾，则认为是价格
    if re.match('^\d+(?:\.\d+|)$', x) is not None:
        return np.float64(re.match('^\d+(?:\.\d+|)$', x).group())
    ## 寻找文本中以数字开始，以,，.结尾的字符串，返回该数字
    if re.match('^(\d+\.\d+)[,，.。]', x) is not None:
        return np.float64(re.match('(^\d+\.\d+)[,，.。]', x).group(1))
    ## 寻找文本中第一个以数字+元的字符串，返回该数字
    if re.search('(\d+(?:\.\d+|))元', x) is not None:
        num = re.search('(?:\d+,|)+\d+', x).group().replace(',','')
        if np.float64(num)>2000:
            return None
        else:
            return np.float64(re.search('(\d+(?:\.\d+|))元', x).group(1))

Inc_text_data['S_INC_INITEXECPRI'] = Inc_text_data['PRICE_DESCRIPTION'].apply(lambda x: f(x))

#### 提取考核/行权期数

In [ ]:
## 提取考核/行权期数，剩余273条无法通过字符串模式匹配获取，其中89条字符串原本就是None
dict_temp = {item[0]:item[1]+1 for item in zip('一二三四五六七八九十',range(10))}
for i in range(1,10):
    dict_temp[str(i)]=i
for i in range(1,10):
    dict_temp[i]=i
Inc_text_data['Exercise_num'] = np.nan
## 剔除空格和<br>
Inc_text_data['S_INC_OPTEXESPECIALCONDITION'] = Inc_text_data['S_INC_OPTEXESPECIALCONDITION'].apply(lambda x: x.replace(' ','') if not pd.isna(x) else None)
Inc_text_data['S_INC_OPTEXESPECIALCONDITION'] = Inc_text_data['S_INC_OPTEXESPECIALCONDITION'].apply(lambda x: x.replace('<br>','') if not pd.isna(x) else None)
Inc_text_data['S_INC_OPTEXESPECIALCONDITION'] = Inc_text_data['S_INC_OPTEXESPECIALCONDITION'].apply(lambda x: x.replace('人民币','') if not pd.isna(x) else None)
for i in range(Inc_text_data.shape[0]):
    x = Inc_text_data['S_INC_OPTEXESPECIALCONDITION'].iloc[i]
    if pd.isna(x):
        continue
    result = re.search('考核年度为(\d{4})年?(?:-|至)(\d{4})年?[一二三四五六七八九十]个会计年度[,，](?:每个会计年度考核一次|分年度进行业绩考核)',x)
    if result is not None:
        Inc_text_data['Exercise_num'].iloc[i] = int(result.group(2))-int(result.group(1))+1
        continue
    result = re.search('(\d{4})年?(?:-|至)(\d{4})年[一二三四五六七八九十]?个?会计年度中?[,，](?:每个会计年度考核一次|分年度进行业绩考核|分年度对公司的业绩指标进行考核)',x)
    if result is not None:
        Inc_text_data['Exercise_num'].iloc[i] = int(result.group(2))-int(result.group(1))+1
        continue
    result = re.search('在行权期的([一二三四五六七八九十])个会计年度中?[,，]分年度进行绩效考核并行权',x)
    if result is not None:
        Inc_text_data['Exercise_num'].iloc[i] = dict_temp[result.group(1)]
        continue
    result = re.search('第.[个次批](?:解除?限售期|解除限售|限售期|归属期|可?行权期|解锁期|考核期|行权/解除限售期|归属/行权期|解锁|归属|行权)',x)
    if result is not None:
        result = re.findall('第.[个次批](?:解除?限售期|解除限售|限售期|归属期|可?行权期|解锁期|考核期|行权/解除限售期|归属/行权期|解锁|归属|行权)',x)
        for j in range(len(result)):
            y = result[j]
            result[j] = re.search('第(.)[个次批](?:解除?限售期|解除限售|限售期|归属期|可?行权期|解锁期|考核期|行权/解除限售期|归属/行权期|解锁|归属|行权)',y).group(1)
            result[j] = dict_temp[result[j]]
        Inc_text_data['Exercise_num'].iloc[i] = dict_temp[np.sort(result)[-1]]
        continue
    result = re.search('第.(?:解除限售期|期行权|期解锁|解锁期|行权期|期)', x)
    if result is not None:
        result = re.findall('第.(?:解除限售期|期行权|期解锁|解锁期|行权期|期)',x)
        for j in range(len(result)):
            y = result[j]
            result[j] = re.search('第(.)(?:解除限售期|期行权|期解锁|解锁期|行权期|期)',y).group(1)
            result[j] = dict_temp[result[j]]
        Inc_text_data['Exercise_num'].iloc[i] = dict_temp[np.sort(result)[-1]]
        continue
    result = re.search('考核年度\d{4}年',x)
    if result is not None:
        result = re.findall('考核年度\d{4}年',x)
        for j in range(len(result)):
            y = result[j]
            result[j] = int(re.search('考核年度(\d{4})年',y).group(1))
        Inc_text_data['Exercise_num'].iloc[i] = np.sort(result)[-1]-np.sort(result)[0]+1
        continue
    result = re.search('第.[批]',x)
    if result is not None:
        result = re.findall('第.[批]',x)
        for j in range(len(result)):
            y = result[j]
            result[j] = re.search('第(.)[批]',y).group(1)
            result[j] = dict_temp[result[j]]
        Inc_text_data['Exercise_num'].iloc[i] = dict_temp[np.sort(result)[-1]]
        continue
    result = re.search('考核年度为(\d{4})年?(?:-|至)(\d{4})年?',x)
    if result is not None:
        Inc_text_data['Exercise_num'].iloc[i] = int(result.group(2))-int(result.group(1))+1
        continue
    result = re.search('分[一二三四五六七八九十]期解锁',x)
    if result is not None:
        result = re.findall('分[一二三四五六七八九十]期解锁',x)
        for j in range(len(result)):
            y = result[j]
            result[j] = re.search('分([一二三四五六七八九十])期解锁',y).group(1)
            result[j] = dict_temp[result[j]]
        Inc_text_data['Exercise_num'].iloc[i] = dict_temp[np.sort(result)[-1]]
        continue
    result = re.search('\d{4}年?[:：]',x)
    if result is not None:
        result = re.findall('\d{4}年?[:：]',x)
        for j in range(len(result)):
            y = result[j]
            result[j] = int(re.search('(\d{4})年?[:：]',y).group(1))
        Inc_text_data['Exercise_num'].iloc[i] = np.sort(result)[-1]-np.sort(result)[0]+1
        continue
## 把大于6小于2的值（实际检查时发现这些条目对应的原字符串格式比较奇怪，约10行）设置为nan
Inc_text_data['S_INC_OPTEXESPECIALCONDITION'][(Inc_text_data['Exercise_num']>6)|(Inc_text_data['Exercise_num']<2)]=None

#### 提取第一个行权期的考核年

In [ ]:
## 提取第一个行权期的考核年
financial_index_dict ={'营业收入':0, '营业收入值':0, '主营业务收入':0, '实现的营业收入':0, '经审计营业收入':0, '收入的平均值':0, '平均营业收入':0, '累计营业收入':0, '营业收入均值':0,
                        '营业收入累计':0, '年平均营业收入':0,'销售收入':0,'经审计的营业收入':0,'收入':0,'经审计的合并营业收入':0,'实现营业收入':0,'合并营业收入':0,'EBIT':0,
                        '年度营业收入':0,'营业利润':0,
                        '净利润':1, '税后利润':1, '归属于上市公司股东扣除非经常性损益的净利润':1, '实现的净利润值':1, '考核净利润':1, '可比净利润':1, '净利润的平均值':1, '实现的净利润':1,
                        '剔除激励成本的净利润':1, '经审计的净利润':1,'经审计净利润':1, '基本每股收益':1, '合并报表净利润':1, '平均净利润':1, '扣除非经常性损益后的净利润':1,
                        '归属于上市公司股东的扣除非经常性损益后的净利润的平均数':1, '净利润平均值的绝对值':1, '净利润平均值':1, '平均归属于母公司股东的净利润':1, '归属于母公司股东的净利润':1,
                        '平均归属于母公司所有者的净利润':1, '母公司的净利润':1, '实现净利润的平均数':1, '扣除非经常性损益后的净利润均值':1, '累计净利润':1,
                        '扣除非经常性损益后的归属于上市公司股东的平均净利润':1, '归属于上市公司股东的净利润累计':1, '平均利润总额':1, '扣除非经常性损益后归属于上市公司股东的净利润平均值':1,
                        '归属于上市公司股东的净利润均值':1, '利润总额均值':1, '净利润累计':1, '归属于上市公司股东的净利润的平均值':1,'扣除激励计划激励成本前归属于上市公司股东的净利润':1,
                        '利润总额':1,'扣除本次激励计划激励成本前归属于上市公司股东的净利润':1, '扣除非经常性损益后归属于上市公司股东的净利润':1,'净利润平均数':1,'归属于上市公司股东净利润':1,
                        '剔除股权激励影响后归属于母公司所有者的净利润':1, '剔除股权激励影响后归属于母公司股东的净利润':1,'归属于母公司股东净利润':1,'归属于上市公司股东的净利润':1,
                        '归属于上市公司股东的扣除非经常性损益后的净利润':1,'归属于母公司的净利润':1,'归属于母公司所有者的净利润':1,'经审计后归属于母公司股东的净利润':1,'经审计后的归属于母公司股东的净利润':1,
                        '归属于上市公司股东的扣除非经常性损益的净利润':1,'实现的归属于上市公司股东的净利润':1,'实现的归属于母公司的净利润':1,'年平均净利润':1,'扣除非经常性损益归属于母公司净利润':1,
                        '扣非后?归母净利润':1,'扣非后净利润':1,'扣除股份支付费用的净利润':1,'归母净利润':1,'剔除商誉减值的净利润':1,'净利润值':1,'扣非净利润':1,'剔除股权激励影响后归属于母公司扣非后净利润':1,
                        '归属上市公司股东的净利润':1,'归属于上市公司股东的扣除非经常性损益净利润':1,'实现归属上市公司股东的扣除非经常性损益后的净利润':1,'扣除非经常性损益后的归属于上市公司股东的净利润':1,
                        '实现的归属于上市公司股东的扣除非经常性损益后的净利润':1,'归属上市公司股东的扣除非经常性损益后的净利润':1,'归属母公司股东的净利润':1,'合并净利润':1,
                        '归属于上市公司股东的扣除非经常性损益后净利润':1,'扣除非经常性损益后归属上市公司股东的净利润':1,'归属于上市公司股东扣除非经常性损益后的净利润':1,'实现净利润':1,
                        '利润':1,'归母扣非净利润':1,'实现的归母扣非净利润':1,'净润':1,'扣除非经常性损益后净利润':1,'经审计净利润值':1,'扣除股份支付费用影响后归属于上市公司股东的净利润':1,
                        '主营业务利润':1,'净利率':1,'扣除非经常性损益和股份支付费用影响后的净利润':1,'扣除非经常性损益的净利润':1,'实现的扣除非经常性损益后的净利润':1,
                        '扣除非经常损益后的归属于母公司股东净利润':1,'归属于上市公司股东的扣除非经常性损益的净利润':1,'净利润均值':1,'扣非后归母净利润':1,'平均净利润值':1,
                        '实现的扣除非经常性损益的净利润':1,
                        'ROE':2, '净资产收益率':2, '加权平均净资产收益率':2, '扣除非经常性损益后的净资产收益率':2, '平均的加权平均净资产收益率':2, '平均归母ROE':2, '扣非后的净资产收益率':2,
                        'ΔEVA':3,'业绩':4,'业绩的平均值':4,'平均值':4}
type_dict = {'增长率':0, '増长率':0, '年增长率':0, '增长比例':0, '增幅':0, '增长幅度':0, '累计增长率':0, '增长':0, '总额':1, '分位值':2, '分位点':2, '增长额':3}
financial_index_dict_keys_pattern = '|'.join(financial_index_dict.keys())
type_dict_pattern = '|'.join(type_dict.keys())+'|'
'''
(公司)xx(?!数字)(年)([-|和]xx(年)(x年))(年度|度)(,，:：)(公司)(的)某指标(的)(复合|年复合)(某type_dict)((和|或|及|且)某指标)(的)(复合|年复合)(某type_dict)(:：)(目标值|触发值)(均)(分别)[(为|(应)不低于|大于等于|高于|>=|达到(或超过)|:|：)xx(%)(万|百万|千万|亿)(元)|为正]      这部分没写(触发值[(为|不低(于)|>=|达到|:|：)xx(%)(万|百万|千万|亿)(元))
(公司)xx(?!数字)(年)([-|和]xx(年)(x年))(年度|度)(,，:：)(公司)(的)某指标(的)(复合|年复合)(某type_dict)((和|或|及|且)某指标)(的)(复合|年复合)(某type_dict)(:：)(目标值|触发值)(均)(分别)(为|(应)不低于|大于等于|高于|>=|达到(或超过)|:|：)前x个会计年度(的)平均水[准|平]
xx(?!数字)(年度|年)(,，:：)(公司)(的)(某指标)(的)(复合|年复合)(某type_dict)(,，:：)[目标值|触发值](为|(应)不低于|大于等于|高于|>=|达到(或超过)|:|：|,|，)xx(%)(万|百万|千万|亿)(元)
(以)(公司)xx(?!数字)(年)([-|和]xx(年)(x年))(年度|度)(公司)(的)(某指标)(的)(复合|年复合)(某type_dict)((和|或|及|且)某指标)(xx)(%)(万|百万|千万|亿)(元)为(固定)[基数|基准|基础|基期](计算)[,，](公司)xx(?!数字)(年)([-|和]xx(年)(x年))(年度|度)(,，:：)(公司)(的)(某指标)(的)(复合|年复合)(某type_dict)((和|或|及|且)某指标)(较基数|与基数相比|较基期)(的)(复合|年复合)(某type_dict)(:：)(目标值|触发值)(均)(分别)[(为|(应)不低(于)|大于等于|高于|>=|达到|:|：)xx(%)(万|百万|千万|亿)(元)|为正]
(以)(公司)xx(?!数字)(年)([-|和]xx(年)(x年))(年度|度)(公司)(的)(某指标)(的)(复合|年复合)(某type_dict)((和|或|及|且)某指标)(xx)(%)(万|百万|千万|亿)(元)为(固定)[基数|基准|基础|基期](计算)[,，](公司)xx(?!数字)(年)([-|和]xx(年)(x年))(年度|度)(,，:：)(公司)(的)(某指标)(的)(复合|年复合)(某type_dict)((和|或|及|且)某指标)(较基数|与基数相比|较基期)(的)(复合|年复合)(某type_dict)(:：)(目标值|触发值)(均)(分别)不低于基数(的xx(%?))
(公司)xx(?!数字)(年)([-|和]xx(年)(x年))(年度|度)(,，:：)(公司)(的)某指标(的)(复合|年复合)(某type_dict)((和|或|及|且)某指标)[较|比|相对于|相比(于)|相较(于)|对于]xx(?!数字)(年)([-|和]xx(年)(x年))(度)(,，)(公司)(的)(某指标)(的)(复合|年复合)(某type_dict)((和|或|及|且)某指标)(的)(复合|年复合)[增长|增长率](:：)(目标值|触发值)(均)(分别)[(为|(应)不低(于)|大于等于|高于|>=|达到|:|：)xx(%)(万|百万|千万|亿)(元)|为正]
'''
target_pattern1 = '(?:公司|)(?:(\d{4}(?!\d+)年?(?:[-和]\d{4}年?(?:[234二两三四五六]年|)|)))(?:度|年度|)[,，:：]?(?:公司|)(?:的|)(?:('+financial_index_dict_keys_pattern+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[或和及且]?(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[:：]?(?:目标值|触发值|)均?(?:分别|)(?:(?:≥|为|应?不低于?|大于等于|高于|达到(?:或超过|)|:|：|)(\d+(?:\.\d+|))(%?)(?:万|百万|千万|亿|)(?:元|)|(为正))'
target_pattern11 = '(?:公司|)(?:(\d{4}(?!\d+)年?(?:[-和]\d{4}年?(?:[234二两三四五六]年|)|)))(?:度|年度|)[,，:：]?(?:公司|)(?:的|)(?:('+financial_index_dict_keys_pattern+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[或和及且]?(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[:：]?(?:目标值|触发值|)均?(?:分别|)(?:≥|为|应?不低于?|大于等于|高于|达到(?:或超过|)|:|：|)前[二两三四]个会计年度的?平均水[平准]'
target_pattern2 = '(\d{4})(?!\d+)(?:年度|年|)[,，:：]?(?:公司|)(?:的|)(?:('+financial_index_dict_keys_pattern+'|))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[,，:：]?(?:目标值|触发值)(?:为|(应)不低于|大于等于|高于|>=|达到(或超过)|:|：|,|，|)(\d+(?:\.\d+|))(%?)(?:万|百万|千万|亿|)(?:元|)'
target_pattern3 = '(?:公司|)(?:(\d{4}(?!\d+)年?(?:[-和]\d{4}年?(?:[234二两三四五六]年|)|)))(?:度|年度|)[,，:：]?(?:公司|)(?:的|)(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[或和及且]?(?:('+financial_index_dict_keys_pattern+'|'+')|)(?:较基数|与基数相比|较基期|)(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[:：]?(?:目标值|触发值|)均?(?:分别|)(?:(?:≥|为|应?不低于?|大于等于|高于|达到(?:或超过|)|:|：|)(\d+(?:\.\d+|))(%?)(?:万|百万|千万|亿|)(?:元|)|(为正))'
target_pattern31 = '(?:公司|)(?:(\d{4}(?!\d+)年?(?:[-和]\d{4}年?(?:[234二两三四五六]年|)|)))(?:度|年度|)[,，:：]?(?:公司|)(?:的|)(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[或和及且]?(?:('+financial_index_dict_keys_pattern+'|'+')|)(?:较基数|与基数相比|较基期|)(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[:：]?(?:目标值|触发值|)均?(?:分别|)不低于?基数(?:的(\d+(?:\.\d+|))(%?)|)'
benchmark_pattern = '(?:以|)(?:公司|)(?:(\d{4}(?!\d+)年?(?:[-和]\d{4}年?(?:[234二两三四五六]年|)|)))(?:度|年度|)(?:公司|)(?:的|)(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[或和及且]?(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))(?:\d+(?:\.\d+|)|)(%?)(?:万|百万|千万|亿|)(?:元|)为(?:固定|)(?:基数|基准|基础|基期)(?:计算|)[,，]?'
target_pattern4 = '(?:公司|)(?:(\d{4}(?!\d+)年?(?:[-和]\d{4}年?(?:[234二两三四五六]年|)|)))(?:度|年度|)[,，:：]?(?:公司|)(?:的|)(?:('+financial_index_dict_keys_pattern+')|)(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[或和及且]?(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))(?:较|比|相[比对较]于?|对于)(?:(\d{4}(?!\d+)年?(?:[-和]\d{4}年?(?:[234二两三四五六]年|)|)))(?:度|)[,，]?(?:公司|)(?:的|)(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[或和及且]?(?:('+financial_index_dict_keys_pattern+'|'+'))(?:的|)(?:复合|年复合|)(?:('+type_dict_pattern+'))[:：]?(?:目标值|触发值|)均?(?:分别|)(?:(?:≥|为|应?不低于?|大于等于|高于|达到(?:或超过|)|:|：|)(\d+(?:\.\d+|))(%?)(?:万|百万|千万|亿|)(?:元|)|(为正))'
target_pattern_list = [target_pattern4,[benchmark_pattern,target_pattern3],[benchmark_pattern,target_pattern31], target_pattern2, target_pattern1, target_pattern11]

def f(text, target_pattern_list):
    if pd.isna(text):
        return
    if re.search('(?:首|第.)[个次批](?:解除?限售期|除限售期|解除限售|限售期|归属期|可?行权期|解锁期|考核期|行权/解除限售期|归属/行权期|解锁|归属|行权)', text) is None:
        return
    text_list = re.split('(?:首|第.)[个次批](?:解除?限售期|除限售期|解除限售|限售期|归属期|可?行权期|解锁期|考核期|行权/解除限售期|归属/行权期|解锁|归属|行权)', text)
    x = text_list[1]
    y=x
    y = re.sub('[（\()][A-Za-z]+[）\)]','',y) # 去除形如'（An）'的字符
    y = re.sub('合并报表中?','',y) # 去除合并报表字符
    while 1: # 把数字中的','去除
        m = re.search('\d+,\d+(?:\.\d+|)(?:万|百万|千万|亿|)元?',y)
        if m:
            mm = m.group()
            y = y.replace(mm,mm.replace(',',''))
        else:
            break
    while 1: # 把形如'（xxxx年）'的字符替换成'xxxx年'
        m = re.search('[（\()]即?\d+年[）\)]',y)
        if m:
            mm = m.group()
            y = y.replace(mm,mm[-6:-1])
        else:
            break
    ## 剔除()里的东西
    for i in range(1,20):
        y = re.sub('[（\()].{1,20}[）\)]','',y)
    for i in range(len(target_pattern_list)):
        target_pattern = target_pattern_list[i]
        if type(target_pattern) is str:
            if i>=3 and re.search('(?:基数|基准|基础|基期|较|比|相[比对较]于?)',y) is not None: # 如果已经匹配完target_pattern4和3则要检查’基数‘，’比‘之类的关键词是否在y中，如果在则直接返回空，防止出现错误匹配的情况
                return
            match_result_list = re.findall(target_pattern, y)
            if len(match_result_list)>0:
                year_list = [match_result_list[i][0] for i in range(len(match_result_list))]
                return np.sort(year_list)[-1]
        elif type(target_pattern) is list:
            match_result = re.search(target_pattern[0], y)
            if match_result is not None:
                y_temp = re.split('(?:基数|基准|基础|基期)',y)[1]
                try:
                    match_result_list = re.findall(target_pattern[1], y_temp)
                    if len(match_result_list)>0:
                        year_list = [match_result_list[i][0] for i in range(len(match_result_list))]
                        return np.sort(year_list)[-1]
                except IndexError:
                    continue
Inc_text_data['TARGET_YEAR'] = Inc_text_data['S_INC_OPTEXESPECIALCONDITION'].apply(lambda x:f(x,target_pattern_list))

In [ ]:
## 把年份变成长度为4的年份字符串
def g(x):
    if pd.isna(x):
        return
    if re.search('[二两三四五六]年', x) is not None:
        return x[5:9]
    if '年' in x:
        return x[-5:-1]
    return x[-4:]
Inc_text_data['TARGET_YEAR'] = Inc_text_data['TARGET_YEAR'].apply(lambda x: g(x))
print(Inc_text_data['TARGET_YEAR'].dropna().apply(lambda x: len(re.search('\d+',x).group())==4).sum())##检查是否都变成长度为4的年份字符串

## 把预案公告日的年份>第一个考核年年份的行设置为nan
def g(x):
    if pd.isna(x['TARGET_YEAR']):
        return
    if x['PREPLAN_ANN_DATE'][:4]>x['TARGET_YEAR']:
        return
    return x['TARGET_YEAR']
Inc_text_data['TARGET_YEAR']=Inc_text_data.apply(lambda x: g(x), axis=1)

In [ ]:
## 剩余部分使用人工填充
Inc_text_data.to_csv('Inc_text_data_human.csv',encoding='gbk',index=0)

In [ ]:
## 这里的Inc_text_data_human.csv是人工填充后的数据
Inc_text_data = pd.read_csv('data preprocess\\Inc_text_data_human.csv',encoding='gbk')
Inc_text_data['S_INC_SEQUENCE'] = Inc_text_data['S_INC_SEQUENCE'].apply(lambda x: '0'+str(x) if len(str(x))==3 else str(x))
Inc_text_data['PREPLAN_ANN_DATE'] = pd.to_datetime(Inc_text_data['PREPLAN_ANN_DATE']).apply(lambda x: str(x)[:10])
Inc_text_data.index = pd.MultiIndex.from_frame(Inc_text_data[['S_INFO_WINDCODE','S_INC_SEQUENCE']],names=[None,None])

temp = pd.read_csv('Inc_text_data_human.csv',encoding='gbk')
Inc_text_data['S_INC_INITEXECPRI'] = temp['S_INC_INITEXECPRI'].copy().values

def g(x, tradedays_list=tradedays_list):
    if pd.isna(x['TARGET_YEAR']):
        return np.nan
    preplan_day = x['PREPLAN_ANN_DATE']
    preplan_day_index = tradedays_list.index(to_tradeday(preplan_day,tradedays_list,n=0))
    preplan_year_last_day = preplan_day[:4]+'12-31'
    preplan_year_last_day_index = tradedays_list.index(to_tradeday(preplan_year_last_day,tradedays_list,n=0))
    target_year = int(x['TARGET_YEAR'])
    return target_year-int(x['PREPLAN_ANN_DATE'][:4])+(preplan_year_last_day_index-preplan_day_index)/250
Inc_text_data['Time_from_preplan_to_target'] = Inc_text_data.apply(lambda x:g(x), axis=1)

In [ ]:
Inc_text_data.to_pickle('data preprocess\\Inc_text_data.pkl')

## 提取激励比例信息

In [ ]:
data_temp = pd.read_pickle('data\\AShareIncQuantityDetails.pkl')

In [ ]:
# 提取股权激励比例的数据
'''
步骤：将数据分为四块：（1）S_INC_NAME含（'其他','激励对象','人员','员工','骨干'）且S_INC_POST含'预留'的认为是预留部分
（2）S_INC_NAME含'其他'且S_INC_POST不含'预留'的认为不是给高管的
（3）S_INC_NAME不含'其他'，认为manager_list(1,2,3,4,5)里的职位名称都是高管(不含子公司、分公司)
（4）剩余的都认为不是给高管的(其中一部分是employee和mid_manager里的)
'''
data_temp['S_INC_SEQUENCE2'] = data_temp['S_INC_SEQUENCE'].copy()
data_temp['S_INC_SEQUENCE'] = data_temp['S_INC_SEQUENCE2'].apply(lambda x: x[:2]+'00' if x is not None else None)
data_temp.index = pd.MultiIndex.from_arrays(data_temp[['S_INFO_WINDCODE','S_INC_SEQUENCE']].values.T)
print((~data_temp.duplicated(['S_INFO_WINDCODE','S_INC_SEQUENCE'])).sum())

In [ ]:
def f1(x, title_list, method=None, title_list_without=None):
    spe_out_list = []
    x_list = re.split('[兼,/及]',x)
    for x in x_list:
        for spe_out in spe_out_list:                            
            if spe_out in x:
                return False
        if method=='accuracy':
            for title in title_list:
                if title == x:
                    return True
            return False
        for title in title_list:                                #遍历每个title
            if title in x:                                      #如果某个title在x里
                if method=='out':                               #如果要排除某些title_out
                    count = 0
                    for out in title_list_without:              #对要排除的title_out进行遍历
                        if out in x:                            #如果存在要排除的title_out在x里
                            count = 1
                            break                               #跳出循环，直接跳到下一个x
                    if count:
                        break
                    else:                                       #如果所有要排除的title_out都不在x里，返回True
                        return True
                else:                                           #如果不需要排除某些title_out，直接返回True
                    return True
    return False

# 新建一个dataframe叫做Inc_concentration
Inc_concentration = data_temp[['S_INFO_WINDCODE','S_INC_SEQUENCE']].copy().drop_duplicates()
Inc_concentration[['Management_Inc_ratio','Nonmanagement_Inc_ratio','Reserve_Inc_ratio','Management_num','Nonmanagement_num']] = 0
# 第一块：S_INC_NAME含'其他'且S_INC_POST含'预留'的
data_temp1 = data_temp[(data_temp['S_INC_NAME'].apply(lambda x: f1(x,['其他','激励对象','人员','员工','骨干']) if x is not None else False))]
data_temp11 = data_temp1[(data_temp1['S_INC_POST'].apply(lambda x: f1(x, ['预留']) if x is not None else False))]
_ = data_temp11.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第一块的股权激励百分比
Inc_concentration.loc[_.index, 'Reserve_Inc_ratio'] = _.values # 计入到Reserve_Inc_ratio列里
# 第二块：S_INC_NAME含'其他'且S_INC_POST不含'预留'的
data_temp12 = data_temp1[~(data_temp1['S_INC_POST'].apply(lambda x: f1(x, ['预留']) if x is not None else False))]
_ = data_temp12.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第二块的股权激励百分比
Inc_concentration.loc[_.index, 'Nonmanagement_Inc_ratio'] = _.values # 计入到Nonmanagement_Inc_ratio列里
def get_num(x):
    num = 0
    for i in range(len(x)):
        result = re.search('\d+(?:,\d+|)',x[i])
        if result is not None:
            num += int(result.group().replace(',',''))
        else:
            num = np.nan
    return num
Inc_concentration.loc[_.index, 'Nonmanagement_num'] += data_temp12.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_NAME'].apply(lambda x: get_num(x)).values # 计算非高管人数
print(data_temp11.shape, data_temp12.shape)

In [ ]:
manager_list = ['高管','高级管理人员','董事会主席','董事会秘书','董事长','董事','董秘','总裁','副总裁','副总载','职委会主席','监事','财务负责人',
                '首席技术官','首席信息官','首席财务官','首席战略官','首席投资官','首席法务官','首席设计官','首席研发官','首席人力资源官','首席科学家',
                '首席风险官','首席运营官','首席应用架构师','首席专家','安全可靠首席专家','首席技术专家','CFO',
                '技术部长','财务部长']
manager_list2 = ['总工程师','总会计师','总经济师','总法律顾问']
manager_list3 = ['委员会']
manager_list4 = ['总经理','副总经理','副总']
manager_list5 = ['财务总监','人力资源总监','行政总监','质量总监','营销总监','业务总监','生产总监','技术总监','客服总监','运营管理总监','研发总监',
                '投资总监','运营总监','设计总监','法务总监','合规总监','审计总监','销售总监']
gcd_list = ['党委书记','党委副书记','书记']
mid_manager_list = ['核心管理人员','中高层管理人员','中层管理人员']
employee_list = ['其他人员','核心业务','核心技术','技术人员','符合激励条件的离职人员','员工','骨干','工作人员']
def f(x, title_list, method=None, title_list_without=None):
    x_list = re.split('[兼,/]',x)
    for x in x_list:
        if method=='accuracy':
            for title in title_list:
                if title == x:
                    return True
            return False
        for title in title_list:                                #遍历每个title
            if title in x:                                      #如果某个title在x里
                if method=='out':                               #如果要排除某些title_out
                    count = 0
                    for out in title_list_without:              #对要排除的title_out进行遍历
                        if out in x:                            #如果存在要排除的title_out在x里
                            count = 1
                            break                               #跳出循环，直接跳到下一个x
                    if count:
                        break
                    else:                                       #如果所有要排除的title_out都不在x里，返回True
                        return True
                else:                                           #如果不需要排除某些title_out，直接返回True
                    return True
    return False
# 第三块：S_INC_NAME不含'其他'，认为manager_list(1,2,3,4,5)里的职位名称都是高管(不含子公司、分公司)
data_temp2 = data_temp[~(data_temp['S_INC_NAME'].apply(lambda x: f1(x, ['其他','激励对象','人员','员工','骨干']) if x is not None else False))]
data_temp21 = data_temp2[(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list, method='out', title_list_without=['子公司','分公司','控股公司','控股企业']) if x is not None else False))]
_ = data_temp21.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第三块的股权激励百分比（manager_list）
Inc_concentration.loc[_.index, 'Management_Inc_ratio'] += _.values # 计入到Management_Inc_ratio列里
Inc_concentration.loc[_.index, 'Management_num'] += data_temp21.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].count().values # 计算高管人数

data_temp2 = data_temp2[~(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list, method='out', title_list_without=['子公司','分公司','控股公司','控股企业']) if x is not None else False))]
data_temp22 = data_temp2[(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list2, method='out', title_list_without=['子公司','分公司','控股公司','控股企业','副']) if x is not None else False))]
_ = data_temp22.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第三块的股权激励百分比（manager_list2）
Inc_concentration.loc[_.index, 'Management_Inc_ratio'] += _.values # 计入到Management_Inc_ratio列里
Inc_concentration.loc[_.index, 'Management_num'] += data_temp22.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].count().values # 计算高管人数 

data_temp2 = data_temp2[~(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list2, method='out', title_list_without=['子公司','分公司','控股公司','控股企业','副']) if x is not None else False))]
data_temp23 = data_temp2[(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list3, method='out', title_list_without=['子公司','分公司','控股公司','控股企业']) if x is not None else False))]
_ = data_temp23.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第三块的股权激励百分比（manager_list3）
Inc_concentration.loc[_.index, 'Management_Inc_ratio'] += _.values # 计入到Management_Inc_ratio列里
Inc_concentration.loc[_.index, 'Management_num'] += data_temp23.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].count().values # 计算高管人数

data_temp2 = data_temp2[~(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list3, method='out', title_list_without=['子公司','分公司','控股公司','控股企业']) if x is not None else False))]
data_temp24 = data_temp2[(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list4, method='out', title_list_without=['子公司','分公司','控股公司','控股企业','部总经理','部副总经理']) if x is not None else False))]
_ = data_temp24.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第三块的股权激励百分比（manager_list4）
Inc_concentration.loc[_.index, 'Management_Inc_ratio'] += _.values # 计入到Management_Inc_ratio列里
Inc_concentration.loc[_.index, 'Management_num'] += data_temp24.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].count().values # 计算高管人数

data_temp2 = data_temp2[~(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list4, method='out', title_list_without=['子公司','分公司','控股公司','控股企业','部总经理','部副总经理']) if x is not None else False))]
data_temp25 = data_temp2[(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list5, method='out', title_list_without=['子公司','分公司','控股公司','控股企业','部','助理']) if x is not None else False))]
_ = data_temp25.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第三块的股权激励百分比（manager_list5）
Inc_concentration.loc[_.index, 'Management_Inc_ratio'] += _.values # 计入到Management_Inc_ratio列里
Inc_concentration.loc[_.index, 'Management_num'] += data_temp25.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].count().values # 计算高管人数
# 第四块：剩余的都认为不是给高管的(其中一部分是mid_manager_list和employee里的)
def get_num(x):
    num = 0
    for i in range(len(x)):
        result = re.search('\d+(?:,\d+|)',x[i])
        if result is not None:
            num += int(result.group().replace(',',''))
        else:
            num += 1
    return num
data_temp2 = data_temp2[~(data_temp2['S_INC_POST'].apply(lambda x: f(x, manager_list5, method='out', title_list_without=['子公司','分公司','控股公司','控股企业','部','助理']) if x is not None else False))]
data_temp26 = data_temp2[(data_temp2['S_INC_POST'].apply(lambda x: f(x, mid_manager_list) if x is not None else False))]
_ = data_temp26.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第四块的股权激励百分比（employee_list）
Inc_concentration.loc[_.index, 'Nonmanagement_Inc_ratio'] += _.values # 计入到Management_Inc_ratio列里
Inc_concentration.loc[_.index, 'Nonmanagement_num'] += data_temp26.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_NAME'].apply(lambda x: get_num(x)).values # 计算非高管人数

data_temp2 = data_temp2[~(data_temp2['S_INC_POST'].apply(lambda x: f(x, mid_manager_list) if x is not None else False))]
data_temp27 = data_temp2[(data_temp2['S_INC_POST'].apply(lambda x: f(x, employee_list) if x is not None else False))]
_ = data_temp27.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第四块的股权激励百分比（employee_list）
Inc_concentration.loc[_.index, 'Nonmanagement_Inc_ratio'] += _.values # 计入到Management_Inc_ratio列里
Inc_concentration.loc[_.index, 'Nonmanagement_num'] += data_temp27.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_NAME'].apply(lambda x: get_num(x)).values # 计算非高管人数

data_temp2 = data_temp2[~(data_temp2['S_INC_POST'].apply(lambda x: f(x, employee_list) if x is not None else False))]
_ = data_temp2.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].sum() # 计算第四块的股权激励百分比（不含employee_list）
Inc_concentration.loc[_.index, 'Nonmanagement_Inc_ratio'] += _.values # 计入到Management_Inc_ratio列里
Inc_concentration.loc[_.index, 'Nonmanagement_num'] += data_temp2.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_NAME'].apply(lambda x: get_num(x)).values # 计算非高管人数

Inc_concentration[['Management_Inc_ratio','Nonmanagement_Inc_ratio','Reserve_Inc_ratio']] /= 100

In [ ]:
print(data_temp21.shape, data_temp22.shape, data_temp23.shape, data_temp24.shape, data_temp25.shape, data_temp26.shape, data_temp27.shape, data_temp2.shape, data_temp26.shape[0]+data_temp27.shape[0]+data_temp2.shape[0])

In [ ]:
Inc_concentration['Sum'] = Inc_concentration[['Management_Inc_ratio','Nonmanagement_Inc_ratio','Reserve_Inc_ratio']].sum(axis=1)
Inc_concentration['Sum_withoutR'] = Inc_concentration[['Management_Inc_ratio','Nonmanagement_Inc_ratio']].sum(axis=1)
Inc_concentration['Reserve_to_sum'] = Inc_concentration['Reserve_Inc_ratio']/Inc_concentration['Sum']
Inc_concentration['Is_missing'] = (Inc_concentration['Sum_withoutR']==0)
Inc_concentration['Is_abnormal'] = ((~Inc_concentration['Sum'].apply(lambda x: 0.65<x<1.002)) | (Inc_concentration['Reserve_to_sum'].apply(lambda x: 0.6<x or pd.isna(x))))
# 将['Management_Inc_ratio','Nonmanagement_Inc_ratio']归一化
Inc_concentration[['Management_Inc_ratio','Nonmanagement_Inc_ratio']] = Inc_concentration[['Management_Inc_ratio','Nonmanagement_Inc_ratio']].div(Inc_concentration['Sum_withoutR'], axis=0)

In [ ]:
# 计算前k大股权激励占比，认为S_INC_NAME含'其他'且有人数的为平均分配
# S_INC_NAME含'其他'但不包含人数的共220个样本，目前对于使用这些样本的Topk_Inc_ratio直接设置为nan
k = 3
Inc_concentration['Topk_Inc_ratio'] = np.nan
#### 先把S_INC_NAME含'其他'且有人数的data_temp12展开
data_temp12 = data_temp1[~(data_temp1['S_INC_POST'].apply(lambda x: f1(x, ['预留']) if x is not None else False))]
data_temp12_bool = data_temp12.apply(lambda x: True if re.search('\d+',x.loc['S_INC_NAME']) is not None else False, axis=1)
data_temp12 = data_temp12[data_temp12_bool]
def g(x):
    x_copy = x.copy()
    num = int(re.search('\d+(?:,\d+|)',x_copy.loc['S_INC_NAME']).group().replace(',',''))
    x_copy.loc['S_INC_TOTALQTYPCT'] = x_copy.loc['S_INC_TOTALQTYPCT']/num
    return pd.DataFrame([x_copy.values for _ in range(num)], columns=x_copy.index, index=pd.MultiIndex.from_tuples([x_copy.name for i in range(num)]))
temp = data_temp12.apply(lambda x: g(x), axis=1)
data_temp12 = pd.concat(temp.values)
#### 然后把展开后的data_temp12和data_temp2拼在一起
data_temp2 = data_temp[~(data_temp['S_INC_NAME'].apply(lambda x: f1(x, ['其他','激励对象','人员','员工','骨干']) if x is not None else False))]
data_temp2 = pd.concat([data_temp12,data_temp2])
#### 最后按照['S_INFO_WINDCODE','S_INC_SEQUENCE']分组，再按照S_INC_TOTALQTYPCT降序排序，取前k个值求和
_temp = data_temp2.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].apply(lambda x: x.sort_values(ascending=False)[:k].sum() if len(x)>0 else np.nan)
Inc_concentration.loc[_temp.index, 'Topk_Inc_ratio'] = _temp.values
Inc_concentration['Topk_Inc_ratio'] /= 100

In [ ]:
# 计算['Management_ratio','Nonmanagement_ratio']
Inc_concentration[['Management_ratio','Nonmanagement_ratio']] = Inc_concentration[['Management_num','Nonmanagement_num']].div(Inc_concentration[['Management_num','Nonmanagement_num']].sum(axis=1), axis=0)
# 计算三种分散度：entropy，ML里的基尼系数，经济学里的基尼系数
def gini_ml(x):
    x = np.array(x, dtype=np.float64)
    if len(x)>0:
        x = x/np.sum(x)
        return 1-np.sum(x**2)
def gini_eco(x):
    x = x[x>0]
    if len(x)>0:
        x = x/np.sum(x)
        x = x.sort_values(ascending=False).cumsum().values
        x = np.array(x, dtype=np.float64)
        x = np.append(0,x)
        return 0.5/(len(x)-1)*np.sum(x[:-1]+x[1:])
def entropy(x):
    x = x[x>0]
    x = np.array(x, dtype=np.float64)
    if len(x)>0:
        x = x/np.sum(x)
        return -np.sum(x*np.log(x)) 
Inc_concentration['Inc_entropy'] = Inc_concentration.apply(lambda x: entropy(x.loc[['Management_Inc_ratio','Nonmanagement_Inc_ratio']].values), axis=1)
Inc_entropy_all = data_temp2.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].apply(lambda x: entropy(x.values))
Inc_concentration.loc[Inc_entropy_all.index, 'Inc_entropy_all'] = Inc_entropy_all.values

Inc_concentration['Inc_gini_ml'] = Inc_concentration.apply(lambda x: gini_ml(x.loc[['Management_Inc_ratio','Nonmanagement_Inc_ratio']].values), axis=1)
Inc_gini_ml_all = data_temp2.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].apply(lambda x: gini_ml(x.values))
Inc_concentration.loc[Inc_gini_ml_all.index, 'Inc_gini_ml_all'] = Inc_gini_ml_all.values

Inc_concentration['Inc_gini_eco'] = Inc_concentration.apply(lambda x: gini_eco(x.loc[['Management_Inc_ratio','Nonmanagement_Inc_ratio']]), axis=1)
Inc_gini_eco_all = data_temp2.groupby(['S_INFO_WINDCODE','S_INC_SEQUENCE'])['S_INC_TOTALQTYPCT'].apply(lambda x: gini_eco(x))
Inc_concentration.loc[Inc_gini_eco_all.index, 'Inc_gini_eco_all'] = Inc_gini_eco_all.values

In [ ]:
Inc_concentration = Inc_concentration[~Inc_concentration['Is_abnormal']]
temp_index = Inc_concentration[Inc_concentration['Nonmanagement_num'].isna()].index
Inc_concentration.loc[temp_index, ['Topk_Inc_ratio','Inc_entropy_all','Inc_gini_ml_all','Inc_gini_eco_all']] = np.nan

In [ ]:
Inc_concentration.to_pickle('data preprocess\\Inc_concentration.pkl')

## 提取财务和价量信息

In [ ]:
## 获取报告期
Report_period_dict = {}
temp_list = ['-03-31','-06-30','-09-30','-12-31']
for year in range(1990,2022,1):
    for season in range(4):
        Report_period_dict[str(year)+temp_list[season]] = (year-1990)*4+season
def get_report_period_num(date, Report_period_dict=Report_period_dict):
    assert date in Report_period_dict.keys(), '日期不在Report_period_dict的keys里'
    return Report_period_dict[date]

In [ ]:
output = open('data preprocess\\Report_period_dict.pkl', 'wb+')
pickle.dump(Report_period_dict, output)
output.close()

In [ ]:
'''
财务数据基本预处理：
获取原始数据->检查股票是否在Asharedescription里，如果不在则剔除->REPORT_PERIOD,ANN_DT日期标准化->排序->
剔除AShareBalanceSheet，AShareIncome上市前的行和退市后的行->设置index->添加REPORT_PERIOD_NUM列和SEASON列->检查不同频率的报告期的连续性
'''
# 获取AShareBalanceSheet和AShareIncome
AShareBalanceSheet = pd.read_pickle('data\\AShareBalanceSheet.pkl')
AShareIncome = pd.read_pickle('data\\AShareIncome.pkl')
AShareBalanceSheet.index = AShareBalanceSheet['S_INFO_WINDCODE'].values
AShareIncome.index = AShareIncome['S_INFO_WINDCODE'].values

print('观察到在AShareBalanceSheet但不在Asharedescription的股票有{}只；在AShareIncome但不在Asharedescription的股票有有{}只'.format(
    len(set(AShareBalanceSheet['S_INFO_WINDCODE'])-set(Asharedescription['S_INFO_WINDCODE'])),
    len(set(AShareIncome['S_INFO_WINDCODE'])-set(Asharedescription['S_INFO_WINDCODE'])))) ## 都是A打头的股票
## 剔除AShareBalanceSheet和AShareIncome中在AShareIncome但不在Asharedescription的股票对应的行(因为后面需要用到上市日期)
temp_stock_list = list(set(AShareIncome['S_INFO_WINDCODE'])&set(Asharedescription['S_INFO_WINDCODE']))
AShareIncome = AShareIncome.loc[temp_stock_list]
temp_stock_list = list(set(AShareBalanceSheet['S_INFO_WINDCODE'])&set(Asharedescription['S_INFO_WINDCODE']))
AShareBalanceSheet = AShareBalanceSheet.loc[temp_stock_list]

# REPORT_PERIOD,ANN_DT日期标准化
AShareBalanceSheet['REPORT_PERIOD'] = pd.to_datetime(AShareBalanceSheet['REPORT_PERIOD']).apply(lambda x:str(x)[:10])
AShareIncome['REPORT_PERIOD'] = pd.to_datetime(AShareIncome['REPORT_PERIOD']).apply(lambda x:str(x)[:10])
AShareBalanceSheet['ANN_DT'] = pd.to_datetime(AShareBalanceSheet['ANN_DT']).apply(lambda x:str(x)[:10])
AShareIncome['ANN_DT'] = pd.to_datetime(AShareIncome['ANN_DT']).apply(lambda x:str(x)[:10])
AShareBalanceSheet['ACTUAL_ANN_DT'] = pd.to_datetime(AShareBalanceSheet['ACTUAL_ANN_DT']).apply(lambda x:str(x)[:10])
AShareIncome['ACTUAL_ANN_DT'] = pd.to_datetime(AShareIncome['ACTUAL_ANN_DT']).apply(lambda x:str(x)[:10])
Asharedescription['S_INFO_LISTDATE'] = pd.to_datetime(Asharedescription['S_INFO_LISTDATE']).apply(lambda x:str(x)[:10])
Asharedescription['S_INFO_DELISTDATE'] = pd.to_datetime(Asharedescription['S_INFO_DELISTDATE']).apply(lambda x:str(x)[:10])

# 排序
AShareBalanceSheet = AShareBalanceSheet.sort_values(['S_INFO_WINDCODE','REPORT_PERIOD','ANN_DT','STATEMENT_TYPE'])
AShareIncome = AShareIncome.sort_values(['S_INFO_WINDCODE','REPORT_PERIOD','ANN_DT','STATEMENT_TYPE'])

## 剔除AShareBalanceSheet，AShareIncome上市前的行和退市后的行；剔除在Asharedescription中S_INFO_LISTDATE列为None的AShareBalanceSheet，AShareIncome行(更换新的数据源后就没有这个问题了)
def f(x):
    REPORT_PERIOD = x.loc['REPORT_PERIOD']
    LISTDATE = x.loc['S_INFO_LISTDATE']
    DELISTDATE = x.loc['S_INFO_DELISTDATE']
    if pd.isna(LISTDATE):
        return False
    if LISTDATE<=REPORT_PERIOD:
        if pd.isna(DELISTDATE) or DELISTDATE>REPORT_PERIOD:
            return True
    return False
AShareBalanceSheet_temp = AShareBalanceSheet[['S_INFO_WINDCODE','REPORT_PERIOD']].merge(Asharedescription[['S_INFO_WINDCODE','S_INFO_LISTDATE','S_INFO_DELISTDATE']],on=['S_INFO_WINDCODE'])
bools = AShareBalanceSheet_temp.swifter.apply(lambda x: f(x),axis=1)
AShareBalanceSheet = AShareBalanceSheet[bools.values]
AShareIncome_temp = AShareIncome[['S_INFO_WINDCODE','REPORT_PERIOD']].merge(Asharedescription[['S_INFO_WINDCODE','S_INFO_LISTDATE','S_INFO_DELISTDATE']],on=['S_INFO_WINDCODE'])
bools = AShareIncome_temp.swifter.apply(lambda x: f(x),axis=1)
AShareIncome = AShareIncome[bools.values]
## 设置index
AShareIncome.index = pd.MultiIndex.from_frame(AShareIncome[['S_INFO_WINDCODE','REPORT_PERIOD','STATEMENT_TYPE']])
AShareBalanceSheet.index = pd.MultiIndex.from_frame(AShareBalanceSheet[['S_INFO_WINDCODE','REPORT_PERIOD','STATEMENT_TYPE']])
AShareIncome.index.names = [None,None,None]
AShareBalanceSheet.index.names = [None,None,None]
## 添加REPORT_PERIOD_NUM列和SEASON列
AShareBalanceSheet['REPORT_PERIOD_NUM'] = AShareBalanceSheet['REPORT_PERIOD'].swifter.apply(lambda x: get_report_period_num(x))
AShareIncome['REPORT_PERIOD_NUM'] = AShareIncome['REPORT_PERIOD'].swifter.apply(lambda x: get_report_period_num(x))
AShareBalanceSheet['SEASON'] = AShareBalanceSheet['REPORT_PERIOD_NUM']%4+1
AShareIncome['SEASON'] = AShareIncome['REPORT_PERIOD_NUM']%4+1

## 检查报告期的连续性
print(set(Inc_data['S_INFO_WINDCODE'])-set(AShareIncome['S_INFO_WINDCODE'].unique()))
print('Inc_data中的所有股票在AShareBalanceSheet和AShareIncome都有')
print(AShareIncome['REPORT_PERIOD'].apply(lambda x:x[5:]).unique())
print(AShareBalanceSheet['REPORT_PERIOD'].apply(lambda x:x[5:]).unique())
print('AShareIncome和AShareBalanceSheet的报告期都在3,6,9,12月')

def get_season(date, method='date'):
    return int((pd.to_datetime(date).month-0.5)/3)+1
def f(x):
    REPORT_PERIOD_NUM_start = x['REPORT_PERIOD_NUM'].iloc[0]
    REPORT_PERIOD_NUM_end = x['REPORT_PERIOD_NUM'].iloc[-1]
    return (REPORT_PERIOD_NUM_end-REPORT_PERIOD_NUM_start)==(len(set(x['REPORT_PERIOD_NUM']))), 
temp = AShareBalanceSheet.groupby('S_INFO_WINDCODE').apply(lambda x: f(x))
print(temp.shape[0]==temp.sum())
temp = AShareIncome.groupby('S_INFO_WINDCODE').apply(lambda x: f(x))
print(temp.shape[0]==temp.sum())
print('报告期在季度维度上不连续，某些上市太早的公司没有每季都公告财报')
def f(x):
    x_copy = x[(x%4==3).values]
    if len(x_copy)==0:
        REPORT_PERIOD_NUM_start = x.iloc[0]
        REPORT_PERIOD_NUM_end = x.iloc[-1]
        return (REPORT_PERIOD_NUM_end-REPORT_PERIOD_NUM_start<=1) or ((REPORT_PERIOD_NUM_end-REPORT_PERIOD_NUM_start==2) and REPORT_PERIOD_NUM_end%4==2 and REPORT_PERIOD_NUM_start%4==0)
    temp = (x_copy-3)/4
    year_start = temp.iloc[0]
    year_end = temp.iloc[-1]
    return (year_end - year_start+1)==(len(set(temp)))
temp = AShareBalanceSheet.groupby('S_INFO_WINDCODE')['REPORT_PERIOD_NUM'].apply(lambda x: f(x))
print(temp.shape[0]==temp.sum())
temp = AShareIncome.groupby('S_INFO_WINDCODE')['REPORT_PERIOD_NUM'].apply(lambda x: f(x))
print(temp.shape[0]==temp.sum())
print('报告期在年维度上连续')
def f(x):
    x_copy = x[(x%2==1).values]
    if len(x_copy)==0:
        REPORT_PERIOD_NUM_start = x.iloc[0]
        REPORT_PERIOD_NUM_end = x.iloc[-1]
        return REPORT_PERIOD_NUM_start==REPORT_PERIOD_NUM_end
    temp = (x_copy-1)/2
    half_year_start = temp.iloc[0]
    half_year_end = temp.iloc[-1]
    return (half_year_end - half_year_start+1)==(len(set(temp)))
temp = AShareBalanceSheet.groupby('S_INFO_WINDCODE')['REPORT_PERIOD_NUM'].apply(lambda x: f(x))
print(temp.shape[0]==temp.sum())
temp = AShareIncome.groupby('S_INFO_WINDCODE')['REPORT_PERIOD_NUM'].apply(lambda x: f(x))
print(temp.shape[0]==temp.sum())
print('报告期在半年度维度上不连续，某些上市太早的公司没有每半年都公告财报')

In [ ]:
'''
关于报告期不连续要先做成连续的，对于BS里的数据，直接使用ffill进行填充，影响不大；而对于IS里的数据，填充方式会对结果更有影响，可选的填充方式如下：
1、使用上一季的数据(或n季的数据的平均值)ffill，2、使用当年已产生的数据的平均（对于第一季要单独处理），3、计算上一季的同比增长率，然后根据去年的数据乘以同比增长率（如果去年也没该数据就很麻烦）
综上，根据计算的可行性，暂定使用方法1
另外，为什么不先根据已有的数据算出财务指标，然后再直接填充财务指标数据呢？因为可能在计算财务指标时存在一个数据缺失，另一个数据不缺失，这样会浪费不缺失的数据
'''
AShareIncome_season = AShareIncome[AShareIncome['STATEMENT_TYPE']=='408002000'][['S_INFO_WINDCODE','REPORT_PERIOD','REPORT_PERIOD_NUM','NET_PROFIT_EXCL_MIN_INT_INC','ANN_DT','ACTUAL_ANN_DT']].reset_index(drop=True)
AShareBalanceSheet_season = AShareBalanceSheet[AShareBalanceSheet['STATEMENT_TYPE']=='408001000'][['S_INFO_WINDCODE','REPORT_PERIOD','TOT_SHRHLDR_EQY_EXCL_MIN_INT','ANN_DT','ACTUAL_ANN_DT']].reset_index(drop=True)
AShareIncome_season = AShareIncome_season[AShareIncome_season['REPORT_PERIOD']>='2005-03-31']
AShareBalanceSheet_season = AShareBalanceSheet_season[AShareBalanceSheet_season['REPORT_PERIOD']>='2005-03-31']
print('经检查，AShareBalanceSheet_season和AShareIncome_season都是只在年报上连续，需要在季度上处理报告期不连续的问题')
## 把值为0的地方设置为nan
AShareIncome_season['NET_PROFIT_EXCL_MIN_INT_INC'][AShareIncome_season['NET_PROFIT_EXCL_MIN_INT_INC']==0] = np.nan
AShareBalanceSheet_season['TOT_SHRHLDR_EQY_EXCL_MIN_INT'][AShareBalanceSheet_season['TOT_SHRHLDR_EQY_EXCL_MIN_INT']==0] = np.nan

## 让报告期连续并填充基本信息
def f(x, Report_period_list, sub_list):
    REPORT_PERIOD_start, REPORT_PERIOD_end = x['REPORT_PERIOD'].iloc[0], x['REPORT_PERIOD'].iloc[-1]
    REPORT_PERIOD_start_index, REPORT_PERIOD_end_index = Report_period_list.index(REPORT_PERIOD_start), Report_period_list.index(REPORT_PERIOD_end)
    x_copy = pd.DataFrame(np.nan, index=Report_period_list[REPORT_PERIOD_start_index:REPORT_PERIOD_end_index+1], columns=x.columns)
    x_copy.loc[x['REPORT_PERIOD'].values] = x.copy().values
    x_copy['REPORT_PERIOD'] = Report_period_list[REPORT_PERIOD_start_index:REPORT_PERIOD_end_index+1]
    x_copy[sub_list] = x_copy[sub_list].ffill()
    return x_copy
AShareIncome_season = AShareIncome_season.groupby('S_INFO_WINDCODE').apply(lambda x: f(x, list(Report_period_dict.keys()),['NET_PROFIT_EXCL_MIN_INT_INC'])).reset_index(drop=True)
AShareIncome_season['S_INFO_WINDCODE'] = AShareIncome_season['S_INFO_WINDCODE'].ffill()
AShareBalanceSheet_season = AShareBalanceSheet_season.groupby('S_INFO_WINDCODE').apply(lambda x: f(x, list(Report_period_dict.keys()),['TOT_SHRHLDR_EQY_EXCL_MIN_INT'])).reset_index(drop=True)
AShareBalanceSheet_season['S_INFO_WINDCODE'] = AShareBalanceSheet_season['S_INFO_WINDCODE'].ffill()

In [ ]:
AShareIncome_season.index = pd.MultiIndex.from_frame(AShareIncome_season[['S_INFO_WINDCODE','REPORT_PERIOD']])
AShareIncome_season.index.names = [None,None]
AShareBalanceSheet_season.index = pd.MultiIndex.from_frame(AShareBalanceSheet_season[['S_INFO_WINDCODE','REPORT_PERIOD']])
AShareBalanceSheet_season.index.names = [None,None]

In [ ]:
## 混合BS表和IS表
financial_sheet_merge_season = AShareIncome_season.merge(AShareBalanceSheet_season, on=['S_INFO_WINDCODE','REPORT_PERIOD'])
financial_sheet_merge_season.index = pd.MultiIndex.from_frame(financial_sheet_merge_season[['S_INFO_WINDCODE','REPORT_PERIOD']])
financial_sheet_merge_season.index.names = [None,None]

## 使用最小的日期作为公告日，几乎对所有数据都对
financial_sheet_merge_season['ANN_DT'] = financial_sheet_merge_season[['ANN_DT_x','ANN_DT_y','ACTUAL_ANN_DT_x','ACTUAL_ANN_DT_y']].apply(lambda x: pd.to_datetime(x)).min(axis=1).apply(lambda x: str(x)[:10])
financial_sheet_merge_season = financial_sheet_merge_season.drop(['ANN_DT_x','ANN_DT_y','ACTUAL_ANN_DT_x','ACTUAL_ANN_DT_y'], axis=1)
financial_sheet_merge_season['ANN_DT-REPORT_PERIOD'] = trade_day_interval(financial_sheet_merge_season['REPORT_PERIOD'],financial_sheet_merge_season['ANN_DT'],tradedays_list,buy_open_sell_close=0)
financial_sheet_merge_season['ANN_DT-REPORT_PERIOD_calendar_day'] = financial_sheet_merge_season[['ANN_DT','REPORT_PERIOD']].apply(lambda x: pd.to_datetime(x)).apply(lambda x: (x['ANN_DT']-x['REPORT_PERIOD']).days, axis=1)

计算公式为
$$
ROE = \frac{扣除非经常性损益后净利润（扣除少数股东损益）}{0.5*（期初股东权益合计(不含少数股东权益)+期末股东权益合计(不含少数股东权益)）}\\
=\frac{\text{NET\_PROFIT\_AFTER\_DED\_NR\_LP}}{0.5*(期初\text{TOT\_SHRHLDR\_EQY\_EXCL\_MIN\_INT}+期末\text{TOT\_SHRHLDR\_EQY\_EXCL\_MIN\_INT})}
$$

In [ ]:
## 计算ROE，对于负的净利润不进行处理，对于负的净资产设置为缺失值，让计算出的ROE为缺失值
def f(x):
    x_copy = x.copy()
    x_copy['TOT_SHRHLDR_EQY_EXCL_MIN_INT'][x_copy['TOT_SHRHLDR_EQY_EXCL_MIN_INT']<0] = np.nan
    temp = x_copy['NET_PROFIT_EXCL_MIN_INT_INC']/(x_copy['TOT_SHRHLDR_EQY_EXCL_MIN_INT'].rolling(2,min_periods=2).mean()) ## 关键参数
    return temp
financial_sheet_merge_season['ROE'] = financial_sheet_merge_season.groupby(['S_INFO_WINDCODE']).apply(lambda x: f(x)).values

In [ ]:
'''
计算每个股票预案公告日时的ROE在行业中的分位点：获得股票行业（如果不存在直接将分位点设置为缺失值）和预案公告日日期->获得该日期对应的报告期，
获取该报告期该行业的所有股票代码->获得这些股票截止到预案公告日时的最新的ROE，即ANN_DT<=预案公告日的ROE->计算分位点
'''
Report_period_array = np.array(list(Report_period_dict.keys()))

Inc_financial_and_market = Inc_data[['S_INFO_WINDCODE','S_INC_SEQUENCE','PREPLAN_ANN_DATE','PREPLAN_ANN_DATE_to_trade','PREPLAN_ANN_DATE_to_trade_next']].copy()
Inc_financial_and_market['PREPLAN_ANN_DATE_to_report_period'] = Inc_financial_and_market.apply(lambda x: Report_period_array[np.sum(Report_period_array<x['PREPLAN_ANN_DATE'])-1], axis=1)

def f(x, financial_sheet_merge_season_temp, method='1'):
    financial_sheet_merge_season_temp = financial_sheet_merge_season_temp.copy()
    financial_sheet_merge_season_temp = financial_sheet_merge_season_temp[financial_sheet_merge_season_temp['ANN_DT']<=x['PREPLAN_ANN_DATE']].groupby(['S_INFO_WINDCODE']).apply(lambda x: x.iloc[-1])
    ROE = financial_sheet_merge_season_temp['ROE']
    if pd.isna(ROE.loc[x['S_INFO_WINDCODE']]):
        return np.nan
    if method=='1':
        return RobustCSZScoreNorm(ROE).loc[x['S_INFO_WINDCODE']]
Inc_financial_and_market['ROE_Norm'] = Inc_financial_and_market.reset_index(drop=True).swifter.apply(lambda x: f(x, financial_sheet_merge_season, method='1'), axis=1).values

In [ ]:
## 获取公司类型
Inc_financial_and_market['Corporate_type'] = Inc_financial_and_market.apply(lambda x: Asharedescription.loc[x['S_INFO_WINDCODE'],'Corporate_type'], axis=1).values

In [ ]:
'''
对于研究应用于实盘的策略，收益率要考虑Buy_day_real和'Sell_day_real_'+str(interval)及其区间收益率
## 获取预案发布日下一交易日的作为买入日；如果该日停牌，则取之后不停牌的第一个交易日作为买入日；
## 如果买入日超过预案发布日interval日，仍然按实际取买入日，只不过之后需要判断买入日是否大于卖出日，如果大于则不买入；
## 如果直到最新交易日或退市仍然停牌，则设置为nan
'''
interval = 60
def f(x, data_temp):
    data_temp = data_temp[data_temp.index>=x['PREPLAN_ANN_DATE_to_trade_next']].dropna()
    if len(data_temp) == 0:
        return np.nan
    return data_temp.index[0]
temp = data.copy()
Inc_financial_and_market['Buy_day_real'] = Inc_financial_and_market.apply(lambda x: f(x, temp.loc[x['S_INFO_WINDCODE']]), axis=1)
Inc_financial_and_market['Price_gap'] = Inc_financial_and_market.apply(lambda x: data.loc[(x['S_INFO_WINDCODE'],x['Buy_day_real']),'adjopen']/data.loc[(x['S_INFO_WINDCODE'],x['Buy_day_real']),'adjpreclose']-1 if not pd.isna(data.loc[(x['S_INFO_WINDCODE'],x['Buy_day_real']),'volume']) else np.nan, axis=1)
Inc_financial_and_market['Buy_day_real-PREPLAN_ANN_DATE_to_trade_next'] = Inc_financial_and_market.apply(lambda x: tradedays_list.index(x['Buy_day_real'])-tradedays_list.index(x['PREPLAN_ANN_DATE_to_trade_next']), axis=1)

In [ ]:
Inc_financial_and_market.to_pickle('data preprocess\\Inc_financial_and_market.pkl')

## 提取label信息

In [ ]:
Inc_label = Inc_financial_and_market[['S_INFO_WINDCODE', 'S_INC_SEQUENCE', 'PREPLAN_ANN_DATE',
            'PREPLAN_ANN_DATE_to_trade', 'PREPLAN_ANN_DATE_to_trade_next','Buy_day_real','Buy_day_real-PREPLAN_ANN_DATE_to_trade_next']].copy()
Inc_label['Sell_day_plan_'+str(interval)] = Inc_label.apply(lambda x: to_tradeday(x['PREPLAN_ANN_DATE_to_trade_next'], tradedays_list, n=interval),axis=1)
Inc_label['Sell_day_plan_'+str(interval)+'-PREPLAN_ANN_DATE_to_trade_next'] = Inc_label.apply(lambda x: tradedays_list.index(x['Sell_day_plan_'+str(interval)])-tradedays_list.index(x['PREPLAN_ANN_DATE_to_trade_next']), axis=1)
def f(x, data_temp, interval):
    data_temp = data_temp[data_temp.index>=x['Sell_day_plan_'+str(interval)]].dropna()
    if len(data_temp) == 0:
        return np.nan
    return data_temp.index[0]
temp = data.copy()
Inc_label['Sell_day_real_'+str(interval)] = Inc_label.apply(lambda x: f(x, temp.loc[x['S_INFO_WINDCODE']], interval), axis=1)
Inc_label['Sell_day_real_'+str(interval)+'-PREPLAN_ANN_DATE_to_trade_next'] = Inc_label.apply(lambda x: tradedays_list.index(x['Sell_day_real_'+str(interval)])-tradedays_list.index(x['PREPLAN_ANN_DATE_to_trade_next']), axis=1)
Inc_label['Buy_sell_interval_real_'+str(interval)] = Inc_label.apply(lambda x: tradedays_list.index(x['Sell_day_real_'+str(interval)])-tradedays_list.index(x['Buy_day_real']), axis=1)
Inc_label[str(interval)+'_days_return'] = Inc_label.apply(lambda x:data.loc[(x['S_INFO_WINDCODE'],x['Sell_day_real_'+str(interval)]),'adjclose']/data.loc[(x['S_INFO_WINDCODE'],x['Buy_day_real']),'adjclose']-1 if not pd.isna(data.loc[(x['S_INFO_WINDCODE'],x['Buy_day_real']),'volume']) and not pd.isna(data.loc[(x['S_INFO_WINDCODE'],x['Sell_day_real_'+str(interval)]),'volume']) else np.nan, axis=1)
Inc_label.to_pickle('data preprocess\\Inc_label.pkl')